In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('scraping-371606-9e78aba82a03.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#
date = 0 #当日は0、前日は1・・・
#date = 1 #当日は0、前日は1・・・
date2 = "08/19"
pmc = 13004064 # 店舗ID
SPREADSHEET_KEY = '1b5P0Nxt0qc343J323yGnhBrdEtPH8tbx7sGOfq8Jvww' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 461 # スロットの1台目
wait = 0.02

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }
headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "500":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "500":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

The version of chrome cannot be detected. Trying with latest driver version


['08/19', '025644', '1', '10000.0', '-6319.0', '1', '1']
['08/19', '025758', '2', '12564.0', '-6619.0', '3', '1']
['08/19', '025522', '3', '769.0', '-1203.0', '0', '0']
['08/19', '025606', '4', '40256.0', '-14444.0', '15', '4']
['08/19', '025608', '5', '4872.0', '-1805.0', '2', '2']
['08/19', '025801', '6', '4872.0', '-4212.0', '0', '0']
['08/19', '025830', '7', '8974.0', '16552.0', '17', '2']
['08/19', '025840', '8', '23077.0', '-902.0', '34', '4']
['08/19', '025642', '9', '14615.0', '-3008.0', '5', '1']
['08/19', '025788', '10', '6667.0', '603.0', '6', '6']
['08/19', '025852', '11', '24615.0', '-7823.0', '22', '2']
['08/19', '025564', '12', '10513.0', '-3610.0', '17', '4']
['08/19', '025389', '13', '22051.0', '-2707.0', '47', '3']
['08/19', '025703', '14', '10256.0', '9931.0', '38', '3']
['08/19', '025709', '15', '4615.0', '2107.0', '8', '3']
['08/19', '025780', '16', '12051.0', '904.0', '22', '5']
['08/19', '025557', '17', '7179.0', '-6018.0', '0', '0']
['08/19', '025579', '18', '23

['08/19', '025364', '153', '37436.0', '-2406.0', '39', '15']
['08/19', '025521', '155', '15385.0', '-2406.0', '14', '6']
['08/19', '025364', '156', '33846.0', '-5416.0', '33', '12']
['08/19', '025521', '157', '14103.0', '11737.0', '26', '5']
['08/19', '025364', '158', '20769.0', '-4814.0', '16', '9']
['08/19', '025404', '159', '8205.0', '-1203.0', '8', '4']
['08/19', '025364', '160', '19487.0', '7223.0', '34', '9']
['08/19', '025765', '161', '20769.0', '3010.0', '31', '11']
['08/19', '025877', '162', '50000.0', '3913.0', '53', '22']
['08/19', '025605', '163', '27436.0', '-5717.0', '21', '5']
['08/19', '025489', '164', '25897.0', '-4212.0', '8', '2']
['08/19', '025579', '165', '5641.0', '-4513.0', '0', '0']
['08/19', '025521', '166', '14359.0', '-2106.0', '13', '6']
['08/19', '025579', '167', '8462.0', '-2106.0', '4', '1']
['08/19', '025765', '168', '20256.0', '-3309.0', '21', '9']
['08/19', '025553', '169', '20513.0', '603.0', '20', '6']
['08/19', '025579', '170', '9231.0', '11135.0', 

['08/19', '025540', '304', '5897.0', '-4814.0', '1', '1']
['08/19', '025767', '305', '16667.0', '-6319.0', '13', '4']
['08/19', '025466', '306', '6923.0', '1806.0', '10', '3']
['08/19', '025607', '307', '8718.0', '-3911.0', '3', '3']
['08/19', '025622', '308', '5128.0', '6621.0', '25', '2']
['08/19', '025699', '309', '10000.0', '-6920.0', '3', '3']
['08/19', '025780', '310', '7692.0', '904.0', '18', '4']
['08/19', '025438', '311', '29744.0', '-6619.0', '14', '5']
['08/19', '025825', '312', '12564.0', '-10231.0', '1', '1']
['08/19', '025775', '313', '20000.0', '3010.0', '13', '4']
['08/19', '025573', '314', '9231.0', '-5115.0', '4', '3']
['08/19', '025699', '315', '11538.0', '-8124.0', '4', '4']
['08/19', '025481', '316', '4359.0', '-2707.0', '1', '1']
['08/19', '025668', '317', '4872.0', '-3008.0', '3', '1']
['08/19', '025795', '318', '10513.0', '3311.0', '15', '3']
['08/19', '025548', '319', '10769.0', '-6619.0', '4', '2']
['08/19', '025812', '320', '30769.0', '-23772.0', '1', '1']
['

['08/19', '025724', '447', '15897.0', '-7522.0', '14', '5']
['08/19', '025328', '448', '13077.0', '-6920.0', '4', '2']
['08/19', '025754', '449', '2821.0', '-1203.0', '3', '1']
['08/19', '025885', '450', '17949.0', '8126.0', '88', '5']
['08/19', '025544', '451', '3846.0', '1506.0', '15', '1']
['08/19', '025544', '452', '2821.0', '-2106.0', '1', '1']
['08/19', '025698', '453', '2051.0', '-1203.0', '2', '1']
['08/19', '025855', '454', '513.0', '-902.0', '0', '0']
['08/19', '025534', '455', '9231.0', '-1203.0', '7', '3']
['08/19', '025848', '456', '14615.0', '-1805.0', '20', '8']
['08/19', '025670', '457', '1795.0', '-1504.0', '0', '0']
['08/19', '025835', '458', '12564.0', '302.0', '46', '7']
['08/19', '025509', '459', '12308.0', '-1203.0', '13', '3']
['08/19', '025697', '460', '2821.0', '1806.0', '8', '1']
['08/19', '025536', '501', '2308.0', '-1805.0', '0', '0']
['08/19', '025687', '502', '2051.0', '-1805.0', '0', '0']
['08/19', '025750', '503', '0', '0', '0', '0']
['08/19', '025875', 

['08/19', '120110', '587', '4971', '-1944.0', '21', '12', '--']
['08/19', '120110', '588', '1955', '-231.0', '11', '5', '--']
['08/19', '120110', '589', '787', '-185.0', '4', '1', '--']
['08/19', '120110', '590', '3904', '-1991.0', '18', '6', '--']
['08/19', '120110', '591', '1265', '1991.0', '16', '8', '--']
['08/19', '120110', '592', '3099', '-1343.0', '15', '4', '--']
['08/19', '120110', '593', '1109', '-139.0', '6', '3', '--']
['08/19', '120110', '594', '4933', '-3194.0', '15', '13', '--']
['08/19', '120110', '595', '709', '-370.0', '2', '3', '--']
['08/19', '120110', '596', '2164', '6250.0', '41', '9', '--']
['08/19', '120110', '597', '1676', '-1250.0', '5', '3', '--']
['08/19', '120110', '598', '3964', '5324.0', '41', '28', '--']
['08/19', '120110', '599', '2029', '4537.0', '34', '4', '--']
['08/19', '120110', '600', '3420', '185.0', '24', '7', '--']
['08/19', '119981', '601', '2611', '-2037.0', '8', '6', '--']
['08/19', '119981', '602', '2903', '-3009.0', '4', '6', '--']
['08/19

['08/19', '120145', '737', '5546', '-1667.0', '0', '7', '41']
['08/19', '120145', '738', '3037', '-556.0', '0', '10', '27']
['08/19', '120145', '739', '1052', '-833.0', '0', '3', '5']
['08/19', '120145', '740', '8785', '5648.0', '0', '17', '123']
['08/19', '120025', '741', '5497', '2778.0', '34', '11', '--']
['08/19', '120025', '742', '285', '-185.0', '1', '0', '--']
['08/19', '120025', '743', '5351', '1759.0', '29', '7', '--']
['08/19', '120025', '744', '394', '-231.0', '1', '0', '--']
['08/19', '120025', '745', '2092', '-370.0', '5', '5', '--']
['08/19', '120025', '746', '317', '-463.0', '0', '0', '--']
['08/19', '120025', '747', '4717', '1343.0', '22', '11', '--']
['08/19', '120025', '748', '2983', '1019.0', '14', '10', '--']
['08/19', '120025', '749', '805', '231.0', '4', '1', '--']
['08/19', '120025', '750', '2823', '648.0', '14', '6', '--']
['08/19', '120025', '751', '2953', '972.0', '14', '8', '--']
['08/19', '120025', '752', '1580', '231.0', '7', '5', '--']
['08/19', '120025', 

{'spreadsheetId': '1b5P0Nxt0qc343J323yGnhBrdEtPH8tbx7sGOfq8Jvww',
 'updates': {'spreadsheetId': '1b5P0Nxt0qc343J323yGnhBrdEtPH8tbx7sGOfq8Jvww',
  'updatedRange': 'S5RAW!F6582:M6621',
  'updatedRows': 40,
  'updatedColumns': 8,
  'updatedCells': 320}}

In [3]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "500":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "500":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['07/29', '025644', '1', '14103.0', '-2707.0', '18', '4']
['07/29', '025758', '2', '11026.0', '8427.0', '20', '3']
['07/29', '025522', '3', '5128.0', '-3309.0', '2', '2']
['07/29', '025606', '4', '43590.0', '-18957.0', '12', '2']
['07/29', '025608', '5', '6923.0', '-3911.0', '1', '1']
['07/29', '025801', '6', '17179.0', '2107.0', '11', '2']
['07/29', '025830', '7', '10000.0', '2408.0', '9', '3']
['07/29', '025840', '8', '0', '0', '1', '1']
['07/29', '025642', '9', '20769.0', '3913.0', '14', '3']
['07/29', '025788', '10', '12308.0', '5719.0', '16', '11']
['07/29', '025609', '11', '6410.0', '-1203.0', '12', '2']
['07/29', '025474', '12', '2308.0', '-1504.0', '1', '1']
['07/29', '025389', '13', '7179.0', '-5115.0', '1', '1']
['07/29', '025703', '14', '15385.0', '-3911.0', '16', '4']
['07/29', '025709', '15', '10513.0', '2107.0', '12', '4']
['07/29', '025489', '16', '6154.0', '-1203.0', '2', '1']
['07/29', '025557', '17', '20000.0', '2107.0', '14', '4']
['07/29', '025579', '18', '9744.0', 

['07/29', '025521', '155', '54103.0', '2408.0', '57', '22']
['07/29', '025364', '156', '1282.0', '-1504.0', '0', '0']
['07/29', '025521', '157', '57179.0', '13844.0', '74', '25']
['07/29', '025364', '158', '11538.0', '603.0', '14', '7']
['07/29', '025404', '159', '0', '0', '0', '0']
['07/29', '025364', '160', '36410.0', '2408.0', '45', '13']
['07/29', '025765', '161', '26667.0', '-8124.0', '18', '11']
['07/29', '025877', '162', '51538.0', '4214.0', '59', '22']
['07/29', '025605', '163', '9231.0', '904.0', '12', '4']
['07/29', '025489', '164', '12308.0', '7825.0', '9', '2']
['07/29', '025579', '165', '11538.0', '-4814.0', '2', '2']
['07/29', '025521', '166', '5128.0', '1205.0', '7', '4']
['07/29', '025579', '167', '4872.0', '1806.0', '3', '1']
['07/29', '025765', '168', '14359.0', '-4814.0', '10', '4']
['07/29', '025553', '169', '36667.0', '-2406.0', '29', '6']
['07/29', '025579', '170', '20513.0', '-7823.0', '4', '3']
['07/29', '025605', '171', '3846.0', '4515.0', '8', '2']
['07/29', '

['07/29', '025767', '305', '4872.0', '3311.0', '10', '1']
['07/29', '025466', '306', '12051.0', '-2707.0', '9', '2']
['07/29', '025607', '307', '2308.0', '-2106.0', '0', '0']
['07/29', '025622', '308', '11538.0', '-8726.0', '3', '3']
['07/29', '025699', '309', '7949.0', '6621.0', '17', '5']
['07/29', '025780', '310', '9487.0', '2709.0', '24', '7']
['07/29', '025438', '311', '10513.0', '-1203.0', '6', '3']
['07/29', '025825', '312', '29744.0', '-1805.0', '17', '7']
['07/29', '025775', '313', '6923.0', '8728.0', '8', '1']
['07/29', '025573', '314', '15897.0', '-4814.0', '8', '5']
['07/29', '025699', '315', '5641.0', '603.0', '9', '4']
['07/29', '025481', '316', '27949.0', '12640.0', '36', '7']
['07/29', '025668', '317', '18718.0', '-12337.0', '2', '2']
['07/29', '025795', '318', '26667.0', '-4212.0', '20', '4']
['07/29', '025548', '319', '5641.0', '-2106.0', '7', '1']
['07/29', '025812', '320', '14872.0', '-9930.0', '2', '2']
['07/29', '025492', '321', '12821.0', '3612.0', '19', '1']
['0

['07/29', '025328', '448', '9487.0', '-902.0', '10', '4']
['07/29', '025754', '449', '0', '0', '0', '0']
['07/29', '025885', '450', '14872.0', '603.0', '53', '4']
['07/29', '025544', '451', '0', '0', '0', '0']
['07/29', '025544', '452', '2821.0', '-2106.0', '1', '1']
['07/29', '025698', '453', '3590.0', '-1203.0', '4', '1']
['07/29', '025855', '454', '4615.0', '3612.0', '14', '3']
['07/29', '025534', '455', '5385.0', '904.0', '6', '3']
['07/29', '025848', '456', '16410.0', '-4814.0', '20', '5']
['07/29', '025670', '457', '8974.0', '-7221.0', '0', '0']
['07/29', '025835', '458', '15897.0', '-6319.0', '32', '11']
['07/29', '025509', '459', '9487.0', '-1504.0', '8', '2']
['07/29', '025697', '460', '5128.0', '-1504.0', '6', '3']
['07/29', '025536', '501', '9231.0', '302.0', '3', '1']
['07/29', '025687', '502', '6410.0', '-4212.0', '1', '1']
['07/29', '025750', '503', '17692.0', '-14143.0', '6', '1']
['07/29', '025875', '504', '6923.0', '1205.0', '7', '2']
['07/29', '025826', '505', '0', '0

['07/29', '120110', '594', '4602', '-4074.0', '13', '6', '--']
['07/29', '120110', '595', '1376', '139.0', '8', '4', '--']
['07/29', '120110', '596', '2245', '-972.0', '10', '5', '--']
['07/29', '120110', '597', '2265', '-694.0', '11', '4', '--']
['07/29', '120110', '598', '3034', '2222.0', '29', '10', '--']
['07/29', '120110', '599', '638', '231.0', '4', '4', '--']
['07/29', '120110', '600', '2799', '3843.0', '34', '9', '--']
['07/29', '119981', '601', '2051', '-2037.0', '1', '8', '--']
['07/29', '119981', '602', '2197', '-2500.0', '1', '6', '--']
['07/29', '120068', '603', '1578', '139.0', '10', '4', '--']
['07/29', '120068', '604', '1494', '-185.0', '8', '5', '--']
['07/29', '120092', '605', '0', '0', '0', '0', '--']
['07/29', '120092', '606', '3141', '-2269.0', '12', '2', '--']
['07/29', '120134', '608', '32', '0', '0', '0', '--']
['07/29', '120134', '609', '92', '139.0', '1', '0', '--']
['07/29', '120134', '610', '33', '0', '0', '0', '--']
['07/29', '120134', '611', '32', '0', '0'

['07/29', '120025', '746', '1105', '-278.0', '3', '2', '--']
['07/29', '120025', '747', '840', '93.0', '3', '1', '--']
['07/29', '120025', '748', '5690', '1759.0', '26', '15', '--']
['07/29', '120025', '749', '3752', '-556.0', '13', '6', '--']
['07/29', '120025', '750', '390', '93.0', '2', '0', '--']
['07/29', '120025', '751', '2151', '-185.0', '7', '8', '--']
['07/29', '120025', '752', '1645', '370.0', '7', '7', '--']
['07/29', '120025', '753', '4552', '-185.0', '16', '7', '--']
['07/29', '120025', '754', '601', '-139.0', '2', '1', '--']
['07/29', '120025', '755', '596', '-139.0', '2', '1', '--']
['07/29', '120025', '756', '1383', '-370.0', '3', '4', '--']
['07/29', '120025', '757', '591', '-324.0', '1', '1', '--']
['07/29', '120025', '758', '1415', '-417.0', '5', '2', '--']
['07/29', '120025', '759', '3337', '-185.0', '13', '6', '--']
['07/29', '120025', '760', '1006', '-509.0', '1', '3', '--']
['07/29', '119916', '761', '5455', '-139.0', '17', '19', '--']
['07/29', '119916', '762', 

{'spreadsheetId': '1b5P0Nxt0qc343J323yGnhBrdEtPH8tbx7sGOfq8Jvww',
 'updates': {'spreadsheetId': '1b5P0Nxt0qc343J323yGnhBrdEtPH8tbx7sGOfq8Jvww',
  'updatedRange': 'S5RAW!F5742:M5781',
  'updatedRows': 40,
  'updatedColumns': 8,
  'updatedCells': 320}}